In [337]:
import pandas as pd
import imageio 
import re
import numpy as np
import os
from pathlib import Path
import tensorflow as tf

np.set_printoptions(threshold=np.inf)
from matplotlib import pyplot as plt

class primusSequence:

    def __init__(self, folder, package):
        self.image = imageio.imread("./data/" + package + "/" + folder + "/" + folder + ".png")
        self.semantic = open("./data/" + package + "/" + folder + "/" + folder + ".semantic").read()
        self.labels = re.findall(r'note\-(.+?)\_', self.semantic)
        
    def flattened_image(self):
        im = self.image
        return(im.flatten())
                                    
        
def init_objs(dataset):
    
    if dataset == 'train':
        ROOT = './data/package_aa/'
        package = 'package_aa'
    elif dataset == 'validate':
        ROOT = './data/package_ab/'
        package = 'package_ab'

    folders = [directory for directory in os.listdir(ROOT) if os.path.isdir(ROOT+directory)]
    objs = []
    
    for f in folders:
        objs.append(primusSequence(f, package))
        
    return(objs)

In [76]:
objs = init_objs('train')

In [146]:
train_labels = ['a', 'b'] * 21797

In [148]:
train_images = np.array(train_images)

/Users/George/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [439]:
def map_pitch(pitch):
    
    m = {"a": 0,
         "h": 1,
         "c1": 2,
         "d1": 3,
         "e1": 4,
         "f1": 5,
         "g1": 6,
         "a1": 7,
         "h1": 8,
         "c2": 9,
         "d2": 10,
         "e2": 11,
         "f2": 12,
         "g2": 13,
         "a2": 14,
         "h2": 15,
         "c3": 16,
         "other": 17}
    
    return(m[pitch])

class ViennaNote:
    
    def __init__(self, fn):
        attr = fn.split('-')
        self.type = attr[0]
        self.time = attr[1]
        self.pitch = attr[2]
        self.image = imageio.imread('./data/originals-resized/' + fn)
        
    def resize_image(self, width, height):
        pass
    
class InputData:
    
    ROOT = './data/originals-resized/' 
    
    def __init__(self, path = ROOT):
        self.files = [file for file in os.listdir(path) if os.path.isfile(path + file) and file[0:4] == 'note']
        self.objs = [ViennaNote(f) for f in self.files]
                
    def training_images(self, path = ROOT):
        images = [o.image for o in self.objs]
        return(np.array(images))
    
    def training_labels(self, path = ROOT):
        notes = [o.pitch for o in self.objs]
        labels = [map_pitch(pitch) for pitch in notes]
        return(np.array(labels))

In [440]:
input_data = InputData()

In [441]:
x = input_data.training_images()

In [442]:
x = np.array(x)

In [443]:
train_images = input_data.training_images()
train_labels = input_data.training_labels()

In [444]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (50, 30)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.softmax),
])

def ctc_loss(y_true, y_pred, input_length, label_length, real_y_true_ts):
    return tf.keras.backend.ctc_batch_cost(real_y_true_ts, y_pred, input_length, label_length)

In [445]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [446]:
model.fit(train_images, 
          train_labels, 
          epochs=30)

Epoch 1/30
103/103 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0271
Epoch 2/30
103/103 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0356
Epoch 3/30
103/103 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0300
Epoch 4/30
103/103 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0245
Epoch 5/30
103/103 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0280
Epoch 6/30
103/103 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0251
Epoch 7/30
103/103 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0327
Epoch 8/30
103/103 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0331
Epoch 9/30
103/103 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0271
Epoch 10/30
103/103 [==============================] - 0s 3ms/st

In [322]:
result

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 5, 12, 21, 32], dtype=int32)>

In [438]:
train_images[1]

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.